# EM synapse mapping

Creating a synaptome from a morphology with spines.

# Authentication

Follow the instructions below to authenticate and select a project to work in.

The morphology-with-spines that is to serve as the input of `Synaptome` generation must be accessible from the project.

In [1]:
import obi_auth

from obi_notebook.get_projects import get_projects
from obi_notebook.get_entities import get_entities

from entitysdk.models import CellMorphology, EMCellMesh, EMDenseReconstructionDataset, Circuit
from entitysdk import Client
from entitysdk._server_schemas import AssetLabel

token = obi_auth.get_token(environment="staging", auth_mode="daf")
project_context = get_projects(token, env="staging")

Device Authentication Required

1. Click on authentication URL
2. Complete authentication in browser
3. Return here when done

Authentication URL:
]8;id=336421;https://staging.openbraininstitute.org/auth/realms/SBO/device?user_code=ZCXS-WWHF\https://staging.openbraininstitute.org/auth/realms/SBO/device?user_code=ZCXS-WWHF]8;;\

   ✓ Authentication completed successfully!


Dropdown(description='Select:', options=(('My first project', {'id': '7a411785-6895-4839-aaa2-d9f76e09875a', '…

# Find a skeletonized morphology as input

We search for morphologies originating from the `IARPA MICrONS mouse`, as they are derived by skeletonization

In [2]:
entity_client = Client(token_manager=token, project_context=project_context, environment="staging")

skeletonized_morphology = entity_client.search_entity(entity_type=CellMorphology, query={
    "subject__name": 'IARPA MICrONS mouse'
}).first()

## Get `pt_root_id`. 
Next, we have to link the skeleton to an originating cell mesh. In the absence of an activity linking the two, we match them via the identifier, `pt_root_id`.

We print the description of the neuron below. Please parse the pt root id from it and fill it in below.

In [3]:
display(skeletonized_morphology.description)

# FILL IN BELOW
pt_root_id = 864691136968109774

'Reconstructed morphology and extracted spines of neuron 864691136968109774.'

# Set up task

Now create a CAVE token.

In [13]:
import caveclient
temporary_client = caveclient.CAVEclient()
temporary_client.auth.get_new_token()

New Tokens need to be acquired by hand. Please follow the following steps:
                1) Go to: https://global.daf-apis.com/auth/api/v1/create_token to create a new token.
                2) Log in with your Google credentials and copy the token shown afterward.
                3a) Save it to your computer with: client.auth.save_token(token="PASTE_YOUR_TOKEN_HERE")
                or
                3b) Set it for the current session only with client.auth.token = "PASTE_YOUR_TOKEN_HERE"
                Note: If you need to save or load multiple tokens, please read the documentation for details.
                Warning! Creating a new token by finishing step 2 will invalidate the previous token!


Your token needs to be pasted below.

In [15]:
from obi_one.scientific.tasks.em_synapse_mapping import EMSynapseMappingTask, EMSynapseMappingSingleConfig
from obi_one.scientific.from_id import cell_morphology_from_id

m = cell_morphology_from_id.CellMorphologyFromID(id_str=str(skeletonized_morphology.id))
task_config = EMSynapseMappingSingleConfig(initialize=EMSynapseMappingSingleConfig.Initialize(
    spiny_neuron=m,
), coordinate_output_root="obi-one-test", cave_token="PASTE_YOUR_TOKEN_HERE")

task = EMSynapseMappingTask(config=task_config)

# We peform a quick check: Does a synaptome of this neuron already exist?

In [ ]:
found_circuits = entity_client.search_entity(entity_type=Circuit, query={
    "name__ilike": f"{pt_root_id}"
}, limit=10).all()
if len(found_circuits) > 0:
    print(f"Found the following already existing entities that may built from the selected morphology with identifier {pt_root_id}:")
    print([_circ.name for _circ in found_circuits])
    print("Are you sure you want to continue?")
else:
    task.execute(entity_client)